# Определение стоимости автомобилей

**Содержание:**  
1. [Импорт данных и общая информация](#load_and_info)  
 1.1. [Импорт библиотек](#import)  
 1.2. [Импорт данных](#read)  
 1.3. [Общая информация](#info)  
    
    
2. [Подготовка данных](#preparation)  
 
 
3. [Обучение моделей](#study)  


4. [Анализ моделей](#analysis)  
 4.1. [Сводная таблица](#union_table)  
 4.2. [Проверка лучшей модели на тестовой выборке](#test)  
 4.3. [Сравнение с dummy  моделью](#dummy)  
 

6. [Общий вывод](#final)  

**Заказчик исследования:** сервис по продаже автомобилей с пробегом.  
**Цель исследования:** построить модель прогнозирования рыночной стоимости автомобиля.  
**Инструменты:** для реализации проекта использованы модели LGBMRegressor, DecisionTreeRegressor и RandomForestRegressor.  

**Особенности проекта:** 
Значение метрики MRSE не должно превышать 2500.
Заказчику важны:
- качество предсказания;
- скорость предсказания;
- время обучения.

**Исходные данные:**  
Исходные данные представлены в датасете autos.csv
В датасете содержатся следующие признаки:  

- DateCrawled — дата скачивания анкеты из базы;
- VehicleType — тип автомобильного кузова;
- RegistrationYear — год регистрации автомобиля;
- Gearbox — тип коробки передач;
- Power — мощность (л. с.);
- Model — модель автомобиля;
- Kilometer — пробег (км);
- RegistrationMonth — месяц регистрации автомобиля;
- FuelType — тип топлива;
- Brand — марка автомобиля;
- Repaired — была машина в ремонте или нет;
- DateCreated — дата создания анкеты;
- NumberOfPictures — количество фотографий автомобиля;
- PostalCode — почтовый индекс владельца анкеты (пользователя);
- LastSeen — дата последней активности пользователя.  

Целевой признак:
- Price — цена (евро).

## Импорт данных и общая информация
<a id="load_and_info"></a>

### Импорт библиотек
<a id="import"></a>

In [1]:
import numpy as np
import pandas as pd

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor

import lightgbm as lgb
import timeit
import ydata_profiling
import datetime


from optuna.distributions import FloatDistribution as floatd
from optuna.distributions import IntDistribution as intd
import optuna
from optuna.integration import OptunaSearchCV

D:\Programs\anaconda3\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\bond_\AppData\Roaming\Python\Python311\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


### Импорт данных
<a id="read"></a>

In [2]:
try:
    df = pd.read_csv('/datasets/autos.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')

### Общая информация
<a id="info"></a>

In [3]:
# df.profile_report()

In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
object_list = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

def inf(df_info):
    print()
    print('Общая информация:')
    display(df_info.info(memory_usage='deep'))
    print('_' * 50)
    print()
    print('Уникальные значения строковых признаков:')
    for i in object_list:
        print('{}:'.format(i))
        print(df_info[i].unique())
        print()
    print('Описательная статистика строковых значений:')
    display(df_info.describe(include='object').T)
    print('_' * 50)
    print()
    print('Описательная статистика числовых значений:')
    display(df_info.describe().T)
    print('_' * 50)
    print()
    print('Процент пропущенных значений:')
    display(df_info.isna().mean()*100)
    print('_' * 50)
    print()
    print('Случайная выборка датафрейма:')
    display(df_info.sample(n = 5, random_state = 0))
    print('_' * 100)

In [6]:
inf(df)


Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes

None

__________________________________________________

Уникальные значения строковых признаков:
VehicleType:
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']

Gearbox:
['manual' 'auto' nan]

Model:
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mi

,count,unique,top,freq
DateCrawled,354369,271174,2016-03-24 14:49:47,7
VehicleType,316879,8,sedan,91457
Gearbox,334536,2,manual,268251
Model,334664,250,golf,29232
FuelType,321474,7,petrol,216352
Brand,354369,40,volkswagen,77013
Repaired,283215,2,no,247161
DateCreated,354369,109,2016-04-03 00:00:00,13719
LastSeen,354369,179150,2016-04-06 13:45:54,17


__________________________________________________

Описательная статистика числовых значений:


...

__________________________________________________

Процент пропущенных значений:


DateCrawled           0.000000
Price                 0.000000
VehicleType          10.579368
RegistrationYear      0.000000
Gearbox               5.596709
Power                 0.000000
Model                 5.560588
Kilometer             0.000000
RegistrationMonth     0.000000
FuelType              9.282697
Brand                 0.000000
Repaired             20.079070
DateCreated           0.000000
NumberOfPictures      0.000000
PostalCode            0.000000
LastSeen              0.000000
dtype: float64

__________________________________________________

Случайная выборка датафрейма:


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
349159,2016-03-29 19:43:50,400,wagon,2002,manual,116,focus,150000,0,gasoline,ford,NaN,2016-03-29 00:00:00,0,66564,2016-03-29 19:43:50
338382,2016-03-12 22:52:22,1300,small,2003,NaN,0,clio,150000,12,petrol,renault,no,2016-03-12 00:00:00,0,44357,2016-03-22 13:16:32
70027,2016-03-27 15:48:19,500,bus,2000,manual,0,other,150000,4,gasoline,peugeot,NaN,2016-03-27 00:00:00,0,53121,2016-04-07 13:17:49
39627,2016-03-10 21:39:18,5200,wagon,2007,manual,125,focus,150000,11,petrol,ford,no,2016-03-10 00:00:00,0,93333,2016-04-06 22:15:21
318707,2016-03-10 23:54:50,9200,sedan,2009,manual,160,insignia,150000,5,gasoline,opel,no,2016-03-10 00:00:00,0,70378,2016-03-29 18:46:45


____________________________________________________________________________________________________


**Вывод:**
1. Датасет состоит из 354369 строк и 16 признаков.  
2. Количество пропущенных значений - 181077 (3.2%). Из них:  
- VehicleType - 37490 (10.6%);  
- Gearbox - 19833 (5.6%);  
- Model - 19705 (5.6%);  
- FuelType - 32895 (9.3%);
- Repaired - 71154 (20.1%).

3. Полных дубликатов - 4. Поскольку отсутствуют id анкеты или пользователей, разместивших объявления, нельзя однозначно определить, относятся ли дублирующиеся анкеты к одним и тем же автомобилям. Целесообразно оставить дубликаты.
4. Чаще других встречаются, цены, лежащие в диапазоне 0-2000;
5. Наиболее количество анкет по типам кузова автомобилей:
- sedan - 91457 (25.8%);
- small - 79831	(22.5%);
- wagon - 65166	(18.4%).
6. Тип коробки передач:
- manual - 268251 (75.7%);
- auto - 66285 (18.7%).
7. В 40225 (11.4%) анкет указано нулевое значение мощности двигателя.
8. Наиболее широко представлены модели:
- golf - 29232 (8.2%);
- 3er - 19761 (5.6%).
9. 238209 автомобилей (67.2%) имеют пробег 150000 км.
10. Наиболее количество анкет по типам топлива:
- petrol - 216352 (61.1%);
- gasoline - 98720 (27.9%).
11. Ремонтировался ли автомобиль:
- False	(247161) 69.7%;
- True (36054) 10.2%.
12. Наиболее количество анкет по брендам:
- volkswagen - 77013 (21.7%);
- opel - 39931 (11.3%);
- bmw - 36914 (10.4%).
13. У 100% анкет отсутствуют приложенные фотографии.
14. Присутствуют аномальные значения признаков RegistrationYear, Power, Price.
15. Корреляция между RegistrationYear и VehicleType - 100%.

## Подготовка данных
<a id="preparation"></a>

Приводим названия признаков к нижнему регистру:

In [7]:
df.columns = map(str.lower, df.columns)

Пропуски присутствуют только у категориальных признаков. Заменяем их на значение "other":

In [8]:
df[['vehicletype', 'fueltype', 'repaired', 'gearbox', 'model']] =\
df[['vehicletype', 'fueltype', 'repaired', 'gearbox', 'model']].fillna('other')

Приводим даты к типу datetime:

In [9]:
df['datecrawled'] = pd.to_datetime(df['datecrawled'], format='%Y%m%d %H:%M:%S.%f')
df['datecreated'] = pd.to_datetime(df['datecreated'], format='%Y%m%d %H:%M:%S.%f')
df['lastseen'] = pd.to_datetime(df['lastseen'], format='%Y%m%d %H:%M:%S.%f')

Обрабатываем аномальные значения registrationyear - оставляем автомобили, зарегистрированные в период c 1930 до даты последнего скачивания анкеты:

In [10]:
min_year = 1930
max_year =  df['datecrawled'].dt.isocalendar().year.astype('int64').max()
print('Количество удаленных строк: {}'.format(df.query('registrationyear < @min_year | registrationyear > @max_year').shape[0]))
df = df.loc[(df['registrationyear'] >= min_year) & (df['registrationyear'] <= max_year)].reset_index(drop=True)

Количество удаленных строк: 14713


In [11]:
df.head()

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,other,1993,manual,0,golf,150000,0,petrol,volkswagen,other,2016-03-24,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,other,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,other,2016-03-14,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,0,60437,2016-04-06 10:17:21


Обрабатываем аномальные значения power - для каждой модели автомобиля заменяем выбросы на медианные значения, оставшиеся нулевые значения удаляем:

In [12]:
# rows_numbers_start = df.shape[0]
# for i in df['model'].unique():
#     df_tmp = df.loc[(df['model'] == i) & (df['power'] != 0)].reset_index(drop=True)
#     try:
#         min = np.percentile(df_tmp['power'], 25)
#         max = np.percentile(df_tmp['power'], 75)
#         df.loc[(df['model'] == i) & ((df['power'] < min) | (df['power'] > max)), 'power'] =\
#         df.loc[(df['model'] == i)]['power'].median()
#     except:
#         print('Не обработана модель {}'.format(i))
# print('Количество удаленных строк с нулевым значением {}: {}'.format('power', df.query('power == 0').shape[0]))
# df = df.loc[df['power'] != 0].reset_index(drop=True)
rows_numbers_start = df.shape[0]
df['power'] = (df
               .groupby(['model'], dropna=False)['power']
               .transform(lambda x: x.mask(x==0, x[x!=0].median() if x[x!=0].count()!=0 else 0))
              )

Удаляем анкеты, у которых значения целевого признака price равны нулю:

In [13]:
df = df.loc[df['price'] >= 100].reset_index(drop=True)

Определяем возраст автомобиля на момент подачи объявления:

In [14]:
df['years_old'] = pd.to_datetime(df['datecreated'], errors='coerce').dt.year.astype('int64') - df['registrationyear']

Удаляем признаки, лишние для решения задачи прогнозирования, чтобы не создавали "шумы". В том числе удаляем признак NumberOfPictures, так как в нем лишь одно уникальное значение. Признаки времени 'registrationmonth', 'datecreated', 'registrationyear' исключаем в виду того, что ввели новый признак, предположительно напрямую влияющий на цену автомобиля - его возраст (years_old).
Признаки datecrawled и lastseen описывают взаимодействие пользователей с платформой и, предположительно, на стоимость влияния не оказывают.

In [15]:
df = df.drop(['datecrawled', 'registrationmonth', 'datecreated', 'postalcode', 'lastseen', 'registrationyear'], axis=1)

В поле model присутствуют 250 уникальных значений. Поскольку редко встречающиеся значения для обучения модели играют незначительную роль, а также с учетом необходимости проведения OHE-кодирования для обучения моделей DecisionTreeRegressor и RandomForestRegressor, оставляем только 10 наиболее часто встречающихся моделей. Остальные заменяем значением "other":

In [16]:
list_trim = list(df['model'].value_counts().reset_index().head(10)['index'])
df.loc[~df['model'].isin(list_trim), 'model'] = 'other'

Разделяем признаки на числовые, категориальные и целевой:

In [17]:
target_feature = 'price'
categorical_features = ['vehicletype', 'gearbox',  'model', 'fueltype', 'brand', 'repaired']
numerical_features = ['power', 'kilometer', 'years_old']

Для обучения модели LGBMRegressor меняем тип категориальных признаков на "category":

In [18]:
df[categorical_features] = df[categorical_features].apply(lambda x: x.astype('category'))

In [19]:
rows_del = rows_numbers_start - df.shape[0]
print('Количество удаленных строк: {} ({:.2%})'.format(rows_del, rows_del / rows_numbers_start))

Количество удаленных строк: 12377 (3.64%)


**Вывод:**  
1. Названия признаков приведены к нижнему регистру.
2. Обработаны анамальные значения признаков registrationyear, power, price.
3. Добавлен признак "years_old" - возраст автомобиля на момент подачи объявления;
4. Удалены признаки, лишние для решения задачи прогнозирования:
- datecrawled;
- registrationmonth;
- datecreated;
- postalcode;
- lastseen;
- registrationyear.

5. Пропущенные значения заменены на "other";
6. Для признака model оставлены только 10 наиболее часто встречающихся значений. Остальные заменены на "other".
7. признаки разделены на числовые, категориальные и целевой.
8. Для категориальных данных тип изменен на "category".
9. Количество удаленных строк: 13196 (3.73%).

## Обучение моделей
<a id="study"></a>

Задаем общий параметр random_state:

In [20]:
RS = np.random.RandomState(5)

Разбиваем датафрейм на обучающую, валидационную и тестовую выборки:

In [21]:
features = df.drop('price', axis=1)
target = df['price']
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.3, random_state=RS)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

Функция обучения модели и ее проверки на валидационной выборке:

In [22]:
def predict_function(model, params, X_train=X_train):
    optuna.logging.set_verbosity(optuna.logging.WARNING)                               # отключаем логгирование
    model = OptunaSearchCV(model, params, scoring='r2', cv=5)                     # передаем данные в OptunaSearchCV
    model.fit(X_train, y_train)
    start_time = timeit.default_timer()
    model.best_estimator_.fit(X_train, y_train)
    time_fit = timeit.default_timer() - start_time
    pred = model.best_estimator_.predict(X_train)
    rmse_valid = int(mean_squared_error(y_train, pred, squared=False))
    r2_valid = model.best_score_
    return model, rmse_valid, r2_valid, time_fit

**Обучение модели LGBMRegressor**

Задаем сетку гиперпараметров для GridSearchCV:

In [49]:
params_lgb = {'learning_rate' : floatd(0.005, 0.5, False, 0.005),         # значения гиперпараметров модели для подбора
              'max_depth' : intd(2, 30, False, 1),
              'num_leaves' : intd(2, 30, False, 1),
              'n_estimators' : intd(50, 250, False, 50)}

Обучаем модель LGBMRegressor:

In [50]:
model_lgb, rmse_lgb, r2_lgb, time_fit_lgb = predict_function(lgb.LGBMRegressor(objective = "rmse",
                                                                               random_state=RS),
                                                             params_lgb)

C:\Users\bond_\AppData\Local\Temp\ipykernel_9528\4246448081.py:3: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  model = OptunaSearchCV(model, params, scoring='r2', cv=5)                     # передаем данные в OptunaSearchCV


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003767 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 419
[LightGBM] [Info] Number of data points in the train set: 183276, number of used features: 9
[LightGBM] [Info] Start training from score 4642.785897
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 418
[LightGBM] [Info] Number of data points in the train set: 183276, number of used features: 9
[LightGBM] [Info] Start training from score 4637.711342
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002719 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002832 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 418
[LightGBM] [Info] Number of data points in the train set: 183276, number of used features: 9
[LightGBM] [Info] Start training from score 4637.711342
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003411 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 420
[LightGBM] [Info] Number of data points in the train set: 183276, number of used features: 9
[LightGBM] [Info] Start training from score 4632.272562
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002684 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003472 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 420
[LightGBM] [Info] Number of data points in the train set: 183276, number of used features: 9
[LightGBM] [Info] Start training from score 4632.272562
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003878 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 420
[LightGBM] [Info] Number of data points in the train set: 183276, number of used features: 9
[LightGBM] [Info] Start training from score 4637.316479
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

**Подготовка данных для обучения моделей DecisionTreeRegressor и RandomForestRegressor**

Обучаем scaler и encoder на обучающей выборке:

In [25]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features])

encoder = OneHotEncoder(handle_unknown='ignore', drop='first')
encoder.fit(X_train[categorical_features])

OneHotEncoder(drop='first', handle_unknown='ignore')

Функция масштабирования и кодирования признаков:

In [26]:
def df_transform(data):
    # Масштабирование
    data = data.reset_index(drop=True)
    data[numerical_features] = scaler.transform(data[numerical_features])
    
    # Прямое кодирование (OHE)
    encoder_df = pd.DataFrame(encoder.transform(data[categorical_features]).toarray())
    data = data.join(encoder_df)
    
    # Удаление ненужных столбцов
    data = data.drop(categorical_features, axis=1)
    
    # Перевод названий всех столбцов в str 
    data.columns = data.columns.astype(str)
    return data

Масштабируем и кодируем данные обучающей, валидационной и тестовой выборок:

In [27]:
X_train_ohe = df_transform(X_train)
X_test_ohe = df_transform(X_test)

**Обучение модели DecisionTreeRegressor**

Задаем сетку гиперпараметров для GridSearchCV:

In [28]:
params_dtr = {'min_samples_leaf' : intd(2, 30, False, 1)}                          # значения гиперпараметров модели для подбора

Обучаем модель DecisionTreeRegressor:

In [29]:
model_dtr, rmse_dtr, r2_dtr, time_fit_dtr, = predict_function(DecisionTreeRegressor(random_state=RS),
                                                              params_dtr,
                                                              X_train_ohe)

C:\Users\bond_\AppData\Local\Temp\ipykernel_9528\4246448081.py:3: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  model = OptunaSearchCV(model, params, scoring='r2', cv=5)                     # передаем данные в OptunaSearchCV


**Обучение модели RandomForestRegressor**

Задаем сетку гиперпараметров для GridSearchCV:

In [30]:
params_rfr = {'min_samples_leaf' : intd(2, 10, False, 4)} 

Обучаем модель RandomForestRegressor:

In [31]:
model_rfr, rmse_rfr, r2_rfr, time_fit_rfr = predict_function(RandomForestRegressor(random_state=RS),
                                                             params_rfr,
                                                             X_train_ohe)

C:\Users\bond_\AppData\Local\Temp\ipykernel_9528\4246448081.py:3: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  model = OptunaSearchCV(model, params, scoring='r2', cv=5)                     # передаем данные в OptunaSearchCV


**Вывод:**  
Обучены и проверены на валидационных выборках модели LGBMRegressor, DecisionTreeRegressor и RandomForestRegressor.  

## Анализ моделей
<a id="analysis"></a>

### Сводная таблица
<a id="union_table"></a>

Собираем результаты в таблицу. Добавляем в нее метрику major_metric - произведение важных для заказчика параметров: rmse, time_fit и time_pred.

In [51]:
df_best_model = pd.DataFrame({'model_type' : ['LGBMRegressor', 'DecisionTreeRegressor', 'RandomForestRegressor'],
                              'rmse' : [rmse_lgb, rmse_dtr, rmse_rfr],
                              'r2' : [r2_lgb, r2_dtr, r2_rfr],
                              'time_fit' : [time_fit_lgb, time_fit_dtr, time_fit_rfr],
                              'model' : [model_lgb, model_dtr, model_rfr]})
df_best_model[['model_type', 'rmse', 'r2', 'time_fit']]

,model_type,rmse,r2,time_fit
0,LGBMRegressor,1471,0.872155,0.588197
1,DecisionTreeRegressor,1604,0.840785,1.555368
2,RandomForestRegressor,1147,0.870710,131.909897


Определяем лучшую модель: значение rmse должно быть меньше 2500, а значение major_metric - минимальное среди всех моделей:

In [61]:
best_model_row = df_best_model.loc[(df_best_model['rmse'] < 2500) & (df_best_model['r2'] == df_best_model['r2'].max())]
best_model = best_model_row.iloc[0, 4]
print('Лучшая модель: {}'.format(best_model.best_estimator_))
print('RMSE лучшей модели: {}'.format(best_model_row.iloc[0, 1]))
print('R2 лучшей модели: {:.3f}'.format(best_model_row.iloc[0, 2]))

Лучшая модель: LGBMRegressor(learning_rate=0.3, max_depth=27, n_estimators=250, num_leaves=30,
              objective='rmse',
              random_state=RandomState(MT19937) at 0x2A9902B0240)
RMSE лучшей модели: 1471
R2 лучшей модели: 0.872


### Проверка лучшей модели на тестовой выборке
<a id="test"></a>

Функция расчёта метрик:

In [62]:
def test_metrics(X_test):
    start_time = timeit.default_timer()
    pred = best_model.best_estimator_.predict(X_test)
    time_pred = (timeit.default_timer() - start_time)
    rmse_test = int(mean_squared_error(y_test, pred, squared=False))
    r2_test = r2_score(y_test, pred).round(4)
    print('RMSE: {}'.format(rmse_test))
    print('R2: {}'.format(r2_test))
    print('Время предсказания: {:.1f}'.format(time_pred))
    return rmse_test, r2_test, time_pred

Рассчитываем метрики лучшей модели на тестовой выборке:

In [63]:
if best_model_row.iloc[0, 0] == 'LGBMRegressor':
    rmse_test, r2_test, time_pred_test = test_metrics(X_test)
else:
    rmse_test, r2_test, time_pred_test = test_metrics(X_test_ohe)

RMSE: 1597
R2: 0.8766
Время предсказания: 0.1


### Сравнение с dummy  моделью 
<a id="dummy"></a>

Обучаем dummy и делаем оценку (по умолчанию для dummy модели используется метрика R2):

In [59]:
dummy = DummyRegressor(strategy="median")
dummy.fit(X_test_ohe, y_test)
r2_dummy = dummy.score(X_test,y_test)
print('R2 dummy модели: {:.4f}'.format(r2_dummy))

R2 dummy модели: -0.1400


Сравниваем метрику R2 лучшей модели и dummy модели:

In [60]:
print('R2 лучшей модели: {:.4f}'.format(r2_test))
print('R2 dummy модели: {:.4f}'.format(r2_dummy))
if r2_test > r2_dummy:
    print("Лучшая модель адекватна.")
else:
    print("Лучшая модель неадекватна.")

R2 лучшей модели: 0.8766
R2 dummy модели: -0.1400
Лучшая модель адекватна.


**Вывод:**  

Обучены модели LGBMRegressor, DecisionTreeRegresso и RandomForestRegressor. Значения целевой метрики RMSE каждой модели удовлетворяют требованию - не более 2500.  
По метрике R2 Лучшая модель - LGBMRegressor:
- LGBMRegressor: 0.872;
- DecisionTreeRegressor: 0.841;
- RandomForestRegressor: 0.871.

Гиперпараметры лучшей модели LGBMRegressor:  
- learning_rate: 0.3;
- max_depth: 27;
- n_estimators: 250;
- num_leaves: 30.
Метрики лучшей модели LGBMRegressor:  
- RMSE: 1471;  
- R2: 0.872.  
Время обучения лучшей модели LGBMRegressor: 0.6 секунд.   

На тестовой выборке данная модель показала следующие результаты:
- RMSE: 1597;  
- R2: 0.877.  
- Время предсказания: 0.01 секунд.


Сравнение данной модели с dummy моделью показало ее адекватность.

## Общий вывод
<a id="final"></a>

**Импорт данных и общая информация**  

1. Датасет состоит из 354369 строк и 16 признаков.  
2. Количество пропущенных значений - 181077 (3.2%). Из них:  
- VehicleType - 37490 (10.6%);  
- Gearbox - 19833 (5.6%);  
- Model - 19705 (5.6%);  
- FuelType - 32895 (9.3%);
- Repaired - 71154 (20.1%).

3. Полных дубликатов - 4. Поскольку отсутствуют id анкеты или пользователей, разместивших объявления, нельзя однозначно определить, относятся ли дублирующиеся анкеты к одним и тем же автомобилям. Целесообразно оставить дубликаты.
4. Чаще других встречаются, цены, лежащие в диапазоне 0-2000;
5. Наиболее количество анкет по типам кузова автомобилей:
- sedan - 91457 (25.8%);
- small - 79831	(22.5%);
- wagon - 65166	(18.4%).
6. Тип коробки передач:
- manual - 268251 (75.7%);
- auto - 66285 (18.7%).
7. В 40225 (11.4%) анкет указано нулевое значение мощности двигателя.
8. Наиболее широко представлены модели:
- golf - 29232 (8.2%);
- 3er - 19761 (5.6%).
9. 238209 автомобилей (67.2%) имеют пробег 150000 км.
10. Наиболее количество анкет по типам топлива:
- petrol - 216352 (61.1%);
- gasoline - 98720 (27.9%).
11. Ремонтировался ли автомобиль:
- False	(247161) 69.7%;
- True (36054) 10.2%.
12. Наиболее количество анкет по брендам:
- volkswagen - 77013 (21.7%);
- opel - 39931 (11.3%);
- bmw - 36914 (10.4%).
13. У 100% анкет отсутствуют приложенные фотографии.
14. Корреляция между RegistrationYear и VehicleType - 100%  


**Подготовка данных**  

1. Названия признаков приведены к нижнему регистру.
2. Обработаны анамальные значения признаков registrationyear, power, price.  
3. Добавлен признак "years_old" - возраст автомобиля на момент подачи объявления;
4. Удалены признаки, лишние для решения задачи прогнозирования:
- datecrawled;
- registrationmonth;
- datecreated;
- postalcode;
- lastseen;
- registrationyear.

5. Пропущенные значения заменены на "other";
6. Для признака model оставлены только 10 наиболее часто встречающихся значений. Остальные заменены на "other".
7. признаки разделены на числовые, категориальные и целевой.
8. Для категориальных данных тип изменен на "category".  
9. Количество удаленных строк: 13196 (3.73%).


**Обучение моделей**  

Обучены и проверены на валидационных выборках модели LGBMRegressor, DecisionTreeRegressor и RandomForestRegressor.

**Анализ моделей**

Обучены модели LGBMRegressor, DecisionTreeRegresso и RandomForestRegressor. Значения целевой метрики RMSE каждой модели удовлетворяют требованию - не более 2500.  
По метрике R2 Лучшая модель - LGBMRegressor:
- LGBMRegressor: 0.872;
- DecisionTreeRegressor: 0.841;
- RandomForestRegressor: 0.871.

Гиперпараметры лучшей модели LGBMRegressor:  
- learning_rate: 0.3;
- max_depth: 27;
- n_estimators: 250;
- num_leaves: 30.
Метрики лучшей модели LGBMRegressor:  
- RMSE: 1471;  
- R2: 0.872.  
Время обучения лучшей модели LGBMRegressor: 0.6 секунд.   

На тестовой выборке данная модель показала следующие результаты:
- RMSE: 1597;  
- R2: 0.877.  
- Время предсказания: 0.01 секунд.


Сравнение данной модели с dummy моделью показало ее адекватность.